**Задача 1:** Изучите как работает базовый reference counting

Создайте строку и исследуйте ее reference count на разных этапах. Верните кортеж: (count_after_creation, count_after_ref, count_after_del)

In [2]:
import sys
import gc
import weakref

In [ ]:
def task1():
    str1 = "Какая-то строка"
    count_after_creation = sys.getrefcount(str1)
    str2 = str1
    count_after_ref = sys.getrefcount(str1)
    del str2
    count_after_del = sys.getrefcount(str1)
    return [count_after_creation, count_after_ref, count_after_del]

# Проверка
print("Задача 1:", task1())

Задача 1: [4, 5, 4]


**Задача 2:** Влияние функций на Reference Counting

Создайте список и передайте его в функцию.
Сравните reference count до и после вызова функции.
Верните кортеж: (count_before_call, count_during_call, count_after_call)

In [ ]:
def task2():

    def process_list(lst):
        return sys.getrefcount(lst)
    
    lst = [1, 2, 3, 4, 5]
    count_before_call = sys.getrefcount(lst)
    count_during_call = process_list(lst)
    count_after_call = sys.getrefcount(lst)
    return [count_before_call, count_during_call, count_after_call]

# Проверка
print("Задача 2:", task2())

Задача 2: [2, 3, 2]


**Задача 3:** Циклические ссылки и Memory Leaks

Создайте два объекта с циклической ссылкой, затем разорвите ее.
Используйте gc для проверки количества собранных объектов.
Верните количество объектов собранных gc после разрыва ссылки.

In [20]:
def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None
    a = Node('a')
    b = Node('b')
    a.ref = b
    b.ref = a
    gc.collect()
    a.ref = None
    b.ref = None
    collected = gc.collect()
    return collected

print("Задача 3:", task3())

Задача 3: 0


**Задача 4:** Сравнение Reference Count для разных типов данных

Сравните reference count для чисел, строк, списков и словарей.
Верните словарь с reference count для каждого типа после создания.

In [3]:
def task4():
    obj1 = 10000
    int_count = sys.getrefcount(obj1)
    obj2 = "Какая-то строка"
    str_count = sys.getrefcount(obj2)
    obj3 = [10000]
    lst_count = sys.getrefcount(obj3)
    obj4 = {10000 : 10000}
    dict_count = sys.getrefcount(obj4)
    return {int : int_count, str : str_count, list : lst_count, dict : dict_count}
print("Задача 4:", task4())

Задача 4: {<class 'int'>: 4, <class 'str'>: 4, <class 'list'>: 2, <class 'dict'>: 2}


**Задача 5:** Weak References

Создайте два объекта со слабой ссылкой (weakref) между ними.
Убедитесь, что объекты могут быть удалены сборщиком мусора.
Верните True если weakref не увеличивает reference count.

In [22]:
def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    a = Data(1000)
    b = Data(2000)
    before = [sys.getrefcount(a), sys.getrefcount(b)]
    a.weak_ref = weakref.ref(b)
    b.weak_ref = weakref.ref(a)
    after = [sys.getrefcount(a), sys.getrefcount(b)]
    del a
    del b
    print(gc.collect()) # удаление прошло успешно
    return before == after

print("Задача 5:", task5())

6
Задача 5: True


**Задача 6:** Мониторинг работы Garbage Collector

Зарегистрируйте callback функцию для отслеживания работы GC.
Верните список событий, которые отследил callback.

In [23]:
def task6():
    events = []

    def callback(phase, info):
        events.append(f"Phase : {phase}, info : {info}")

    gc.callbacks.append(callback)
    for i in range(5):
        current = [i] * (i + 1)
        del current
    gc.collect()
    gc.callbacks.remove(callback)
    return events   

print("Задача 6:", task6())

Задача 6: ["Phase : start, info : {'generation': 2, 'collected': 0, 'uncollectable': 0}", "Phase : stop, info : {'generation': 2, 'collected': 6, 'uncollectable': 0}"]


**Задача 7:** Анализ поколений в GC

Создайте объекты и отследите их перемещение между поколениями GC.
Верните кортеж с количеством объектов в каждом поколении до и после создания объектов.

In [17]:
def task7():
    info = []
    gc.collect()
    info.append(gc.get_count())
    objects = []
    for i in range(10000):
        objects.append([i] * 10)
    info.append(gc.get_count())
    return info

print("Задача 7:", task7())

Задача 7: [(2, 0, 0), (102, 2, 1)]


**Задача 8:** Мониторинг порогов сборки мусора

Изучите как меняются счетчики поколений GC при создании объектов.
Верните словарь с состоянием счетчиков до и после создания объектов.

In [24]:
def task8():
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()
    
    objects = []
    for i in range(10000):
        objects.append([i] * 10)
    
    after_creation_count = gc.get_count()
    
    collected = gc.collect(0)
    after_collect_count = gc.get_count()
    
    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }

print("Задача 8:", task8())

Задача 8: {'thresholds': (700, 10, 10), 'initial_count': (392, 0, 0), 'after_creation': (494, 2, 1), 'after_collect_gen0': (0, 3, 1), 'collected_objects': 0}


**Задача 9:** Быстрая проверка работы GC

Проверьте, собирает ли GC циклические ссылки.
Верните разницу в количестве объектов до и после сборки.

In [25]:
def task9():
    a, b, c = [], [], []
    a.append(b)
    b.append(c)
    c.append(a)
    
    count_before = len(gc.get_objects())
    collected = gc.collect()
    count_after = len(gc.get_objects())
    
    return count_before - count_after, collected

print("Задача 9:", task9())

Задача 9: (47, 0)


**Задача 10:** Детектор "неубиваемых" объектов

Найдите объекты, которые переживают принудительную сборку мусора.
Выведите количество таких объектов.

In [ ]:
def task10():
    # Небольшая подсказка для создания неудаляемых объектов
    class Persistent:
        def __del__(self):
            pass  # Мешает нормальному удалению

    persistent_objects = []
    for i in range(100):
        persistent_objects.append(Persistent()) # бессмертные
    
    normal_objects = []
    for i in range(100):
        normal_objects.append([i]) # смертные
    
    gc.collect()
    
    del normal_objects
    del persistent_objects
    
    count_after = len(gc.get_objects())
    
    return count_after

print("Задача 10:", task10())

Задача 10: 99199
